### **IMPORTING LIBRARIES**

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('FAOSTAT.csv')

### **EXPLORATORY DATA ANALYSIS:-**

In [3]:
df.head()

,Domain,Area,Element,Item,Year,Unit,Value
0,Food Balances (2010-),Pakistan,Food supply quantity (kg/capita/yr),Wheat and products,2015,kg/cap,104.15
1,Food Balances (2010-),Pakistan,Food supply quantity (kg/capita/yr),Wheat and products,2016,kg/cap,103.76
2,Food Balances (2010-),Pakistan,Food supply quantity (kg/capita/yr),Wheat and products,2017,kg/cap,104.15
3,Food Balances (2010-),Pakistan,Food supply quantity (kg/capita/yr),Wheat and products,2018,kg/cap,99.40
4,Food Balances (2010-),Pakistan,Food supply quantity (kg/capita/yr),Wheat and products,2019,kg/cap,93.57


In [4]:
df.tail()

,Domain,Area,Element,Item,Year,Unit,Value
539,Food Balances (2010-),Pakistan,Food supply quantity (kg/capita/yr),Miscellaneous,2018,kg/cap,0.05
540,Food Balances (2010-),Pakistan,Food supply quantity (kg/capita/yr),Miscellaneous,2019,kg/cap,0.04
541,Food Balances (2010-),Pakistan,Food supply quantity (kg/capita/yr),Miscellaneous,2020,kg/cap,0.01
542,Food Balances (2010-),Pakistan,Food supply quantity (kg/capita/yr),Miscellaneous,2021,kg/cap,0.00
543,Food Balances (2010-),Pakistan,Food supply quantity (kg/capita/yr),Miscellaneous,2022,kg/cap,0.00


In [5]:
df.describe()

,Year,Value
count,544.000000,544.000000
mean,2018.500000,6.021452
std,2.293397,18.421283
min,2015.000000,0.000000
25%,2016.750000,0.040000
50%,2018.500000,0.410000
75%,2020.250000,3.232500
max,2022.000000,120.740000


In [6]:
df.columns

Index(['Domain', 'Area', 'Element', 'Item', 'Year', 'Unit', 'Value'], dtype='object')

In [7]:
df.shape

(544, 7)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544 entries, 0 to 543
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Domain   544 non-null    object 
 1   Area     544 non-null    object 
 2   Element  544 non-null    object 
 3   Item     544 non-null    object 
 4   Year     544 non-null    int64  
 5   Unit     544 non-null    object 
 6   Value    544 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 29.9+ KB


In [9]:
df.isnull().sum()

,0
Domain,0
Area,0
Element,0
Item,0
Year,0
Unit,0
Value,0


In [10]:
df.duplicated().sum()

np.int64(0)

In [11]:
df.dtypes

,0
Domain,object
Area,object
Element,object
Item,object
Year,int64
Unit,object
Value,float64


### **DATA PREPROCESSING:**

In [12]:
#Drop irrelevant columns
df.drop(columns=['Domain', 'Area', 'Element', 'Unit'], inplace=True)

#Rename columns
df.rename(columns={'Item': 'Food_Item', 'Value': 'Per_Capita_kg', 'Year': 'Year'}, inplace=True)

In [13]:
df.head()

,Food_Item,Year,Per_Capita_kg
0,Wheat and products,2015,104.15
1,Wheat and products,2016,103.76
2,Wheat and products,2017,104.15
3,Wheat and products,2018,99.40
4,Wheat and products,2019,93.57


In [14]:
#Pivot table for Years as rows and Items as columns
pivot_df = df.pivot(index='Year', columns='Food_Item', values='Per_Capita_kg')
#Reset index
pivot_df.reset_index(inplace=True)
pivot_df

Food_Item,Year,Apples and products,"Aquatic Animals, Others",Bananas,Barley and products,Beans,"Beverages, Fermented",Bovine Meat,"Butter, Ghee",Cassava and products,...,Sugar (Raw Equivalent),Sugar cane,Sugar non-centrifugal,Sunflowerseed Oil,Sweet potatoes,"Sweeteners, Other",Tea (including mate),Tomatoes and products,"Vegetables, other",Wheat and products
0,2015,2.85,0.01,0.26,0.28,0.84,0.16,8.02,5.00,0.00,...,21.39,1.35,1.62,0.04,0.04,0.34,0.76,3.74,12.93,104.15
1,2016,3.23,0.01,0.34,0.39,1.03,0.16,8.25,4.84,0.00,...,23.62,0.51,1.85,0.00,0.05,0.34,0.85,3.64,13.83,103.76
2,2017,2.95,0.01,0.37,0.24,0.99,0.15,9.39,4.92,0.00,...,22.26,1.14,1.01,0.00,0.05,0.32,0.83,2.72,15.54,104.15
3,2018,2.35,0.01,0.27,0.26,0.89,0.16,9.56,4.99,0.00,...,23.63,0.93,0.91,0.00,0.05,0.29,0.89,3.06,15.91,99.40
4,2019,2.74,0.01,0.04,0.23,0.91,0.16,9.65,5.32,0.00,...,20.64,0.60,1.09,0.00,0.06,0.34,0.89,3.31,15.86,93.57
5,2020,3.05,0.01,0.16,0.19,1.46,0.15,9.44,5.11,0.00,...,26.02,4.54,1.29,0.00,0.06,0.32,0.92,3.96,21.36,101.89
6,2021,2.95,0.01,0.32,0.16,1.52,0.02,9.94,5.17,0.02,...,25.59,4.36,2.11,0.00,0.06,0.39,1.09,4.85,22.66,103.99
7,2022,2.99,0.01,0.29,0.14,1.01,0.02,10.07,5.22,0.02,...,31.71,1.11,0.82,0.00,0.06,0.32,1.04,4.52,18.54,102.95


In [15]:
#Defining food categories for grouping
food_categories = {
    'Cereals': ['Wheat and products', 'Rice and products', 'Barley and products', 'Maize and products',
                'Millet and products', 'Sorghum and products', 'Cereals, other'],
    'Roots & Tubers': ['Cassava and products', 'Potatoes and products', 'Sweet potatoes'],
    'Sugars & Sweeteners': ['Sugar non-centrifugal', 'Sugar (Raw Equivalent)', 'Sweeteners, Other'],
    'Pulses': ['Beans', 'Peas', 'Pulses, Other and products'],
    'Nuts & Oilseeds': ['Nuts and products', 'Groundnuts', 'Coconuts - Incl Copra'],
    'Oils': ['Soyabean Oil', 'Groundnut Oil', 'Sunflowerseed Oil', 'Palm Oil', 'Sesameseed Oil',
             'Olive Oil', 'Ricebran Oil', 'Maize Germ Oil', 'Oilcrops Oil, Other'],
    'Vegetables': ['Tomatoes and products', 'Onions', 'Vegetables, other'],
    'Fruits': ['Oranges, Mandarines', 'Lemons, Limes and products', 'Citrus, Other', 'Bananas',
               'Apples and products', 'Pineapples and products', 'Dates', 'Fruits, other'],
    'Beverages': ['Coffee and products', 'Cocoa Beans and products', 'Tea (including mate)',
                  'Beverages, Fermented'],
    'Spices': ['Pepper', 'Pimento', 'Cloves', 'Spices, Other'],
    'Meat & Animal Products': ['Bovine Meat', 'Mutton & Goat Meat', 'Poultry Meat', 'Meat, Other',
                               'Offals, Edible', 'Butter, Ghee', 'Cream', 'Fats, Animals, Raw', 'Eggs',
                               'Milk - Excluding Butter'],
    'Fish, Seafood': ['Freshwater Fish', 'Demersal Fish', 'Pelagic Fish', 'Crustaceans', 'Cephalopods',
                       'Molluscs, Other', 'Aquatic Animals, Others'],
    'Other': ['Infant food', 'Miscellaneous']
}


In [16]:
#Adding a Category column to the dataframe
def assign_category(food_item):
    for category, items in food_categories.items():
        if food_item in items:
            return category
    return 'Other'
df['Category'] = df['Food_Item'].apply(assign_category)

In [17]:
#Aggregate data by Category and Year
category_trend = df.groupby(['Year', 'Category'])['Per_Capita_kg'].sum().reset_index()

In [18]:
df.head()

,Food_Item,Year,Per_Capita_kg,Category
0,Wheat and products,2015,104.15,Cereals
1,Wheat and products,2016,103.76,Cereals
2,Wheat and products,2017,104.15,Cereals
3,Wheat and products,2018,99.40,Cereals
4,Wheat and products,2019,93.57,Cereals


### **DATA VISUALIZATION**

#### **Per Capita Food Supply Trends by Category (2015–2022)**

In [19]:
fig = go.Figure()
categories = category_trend['Category'].unique()
colors = px.colors.qualitative.Plotly
for i, category in enumerate(categories):
    cat_data = category_trend[category_trend['Category'] == category]
    fig.add_trace(
        go.Scatter(
            x=cat_data['Year'],
            y=cat_data['Per_Capita_kg'],
            name=category,
            line=dict(color=colors[i % len(colors)]),
            hovertemplate=(
                '<b>%{fullData.name}</b><br>' +
                'Year: %{x}<br>' +
                'Food Supply: %{y:.2f} kg/capita/yr<br>'
            ),
            hoverlabel=dict(
                font_color='black',
                bgcolor='white',
                bordercolor='black',
                font_size=12
            ),
            visible=True
        )
    )
#dropdown menu
buttons = [
    dict(
        label='All Categories',
        method='update',
        args=[{'visible': [True] * len(categories)},
              {'title': 'Per Capita Food Supply Trends by Category (2015–2022)'}]
    )
]
for category in categories:
    visible = [cat == category for cat in categories]
    buttons.append(
        dict(
            label=category,
            method='update',
            args=[{'visible': visible},
                  {'title': f'Per Capita Food Supply Trends for {category} (2015-2022)'}]
        )
)
fig.update_layout(
    title='Per Capita Food Supply Trends by Category (2015–2022)',
    xaxis_title='Year',
    yaxis_title='Food Supply (kg/capita/yr)',
    xaxis=dict(
        tickmode='linear',
        range=[2014.5, 2022.5]
    ),
    yaxis=dict(gridcolor='rgba(200, 200, 200, 0.2)'),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.4,
        xanchor='center',
        x=0.5,
        font=dict(size=10),
        bgcolor='rgba(255, 255, 255, 0.5)'
    ),
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.95,
            xanchor='right',
            y=1.15,
            yanchor='top'
        )
    ],
    plot_bgcolor='rgba(240, 240, 240, 0.9)',
    paper_bgcolor='rgba(255, 255, 255, 1)',
    font=dict(color='black', size=12),
    showlegend=True,
    hovermode='closest'
)
fig.show()

#### **Top Food Categories by Per Capita Food Supply Across Years**

In [20]:
fig_all = px.bar(
    category_trend,
    y='Category',
    x='Per_Capita_kg',
    animation_frame='Year',
    title='Top Food Categories by Per Capita Food Supply Across Years (2015–2022)',
    labels={'Per_Capita_kg': 'Food Supply (kg/capita/yr)', 'Category': 'Food Category'},
    color='Category',
    color_discrete_sequence=px.colors.qualitative.Vivid,
    orientation='h',
    template='plotly_white'
)
fig_all.update_traces(
    hovertemplate=(
        '<b>%{y}</b><br>' +
        'Year: %{customdata[0]}<br>' +
        'Food Supply: %{x:.2f} kg/capita/yr'
    ),
    hoverlabel=dict(
        font_color='black',
        bgcolor='white',
        bordercolor='black',
        font_size=12
    )
)
fig_all.update_layout(
    yaxis=dict(autorange='reversed', tickfont=dict(size=10)),
    xaxis=dict(gridcolor='rgba(200, 200, 200, 0.2)', title_font=dict(color='black'), tickfont=dict(color='black')),
    plot_bgcolor='rgba(240, 240, 240, 0.9)',
    paper_bgcolor='rgba(255, 255, 255, 1)',
    font=dict(color='black', size=12),
    showlegend=False,
    hovermode='closest',
    transition=dict(duration=300, easing='cubic-in-out')
)
fig_all.show()

#### **Variability in Per Capita Food Supply by Category**

In [21]:
fig = go.Figure()
categories = df['Category'].unique()
colors = px.colors.qualitative.Pastel
for i, category in enumerate(categories):
    cat_data = df[df['Category'] == category]
    fig.add_trace(
        go.Box(
            y=cat_data['Per_Capita_kg'],
            x=[category] * len(cat_data),
            name=category,
            marker_color=colors[i % len(colors)],
            hovertemplate=(
                '<b>%{fullData.name}</b><br>' +
                'Median: %{median:.2f} kg/capita/yr<br>' +
                'Q1: %{q1:.2f} kg/capita/yr<br>' +
                'Q3: %{q3:.2f} kg/capita/yr<br>' +
                'Min: %{lowerfence:.2f} kg/capita/yr<br>' +
                'Max: %{upperfence:.2f} kg/capita/yr<br>'
            ),
            hoverlabel=dict(
                font_color='white',
                bgcolor='rgba(0, 0, 0, 0.8)',
                bordercolor='white',
                font_size=12
            ),
            visible=True
        )
    )
#dropdown menu
buttons = [
    dict(
        label='All Categories',
        method='update',
        args=[{'visible': [True] * len(categories)},
              {'title': 'Variability in Per Capita Food Supply by Category (2015–2022)'}]
    )
]
for category in categories:
    visible = [cat == category for cat in categories]
    buttons.append(
        dict(
            label=category,
            method='update',
            args=[{'visible': visible},
                  {'title': f'Variability in Per Capita Food Supply by {category} (2015-2022)'}]
        )
)
fig.update_layout(
    title='Variability in Per Capita Food Supply by Category (2015–2022)',
    xaxis_title='Food Category',
    yaxis_title='Food Supply (kg/capita/yr)',
    xaxis=dict(
        tickangle=0
    ),
    yaxis=dict(gridcolor='rgba(200, 200, 200, 0.2)'),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.4,
        xanchor='center',
        x=0.5,
        font=dict(size=10),
        bgcolor='rgba(255, 255, 255, 0.5)'
    ),
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.95,
            xanchor='right',
            y=1.15,
            yanchor='top'
        )
    ],
    plot_bgcolor='rgba(240, 240, 240, 0.9)',
    paper_bgcolor='rgba(255, 255, 255, 1)',
    font=dict(color='black', size=12),
    showlegend=True,
    hovermode='x'
)
fig.show()

#### **Cumulative Per Capita Food Supply by Category Over Time**

In [22]:
category_trend = df.groupby(['Year', 'Category'])['Per_Capita_kg'].sum().unstack().fillna(0)
fig = go.Figure()
colors = px.colors.qualitative.Pastel
for i, category in enumerate(category_trend.columns):
    fig.add_trace(
        go.Scatter(
            x=category_trend.index,
            y=category_trend[category],
            name=category,
            stackgroup='one',
            line=dict(width=0),
            fill='tonexty',
            fillcolor=colors[i % len(colors)],
            hoverinfo='y+name',
            hovertemplate=(
                '<b>%{data.name}</b><br>' +
                'Year: %{x}<br>' +
                'Food Supply: %{y:.2f} kg/capita/yr<br>'
            ),
            hoverlabel=dict(
                font_color='black',
                bgcolor='white',
                bordercolor='black',
                font_size=12
            )
        )
    )
fig.update_layout(
    title='Cumulative Per Capita Food Supply by Category Over Time (2015–2022)',
    xaxis_title='Year',
    yaxis_title='Food Supply (kg/capita/yr)',
    xaxis=dict(
        tickmode='linear',
        range=[2014.5, 2022.5]
    ),
    yaxis=dict(gridcolor='rgba(200, 200, 200, 0.2)'),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.4,
        xanchor='center',
        x=0.5,
        font=dict(size=10),
        bgcolor='rgba(255, 255, 255, 0.5)'
    ),
    plot_bgcolor='rgba(240, 240, 240, 0.9)',
    paper_bgcolor='rgba(255, 255, 255, 1)',
    font=dict(color='black', size=12),
    showlegend=True,
    hovermode='closest'
)
fig.show()

#### **Per Capita Food Supply Across Categories and Years**

In [23]:
category_data = df.groupby(['Category', 'Year'])['Per_Capita_kg'].sum().reset_index()
pivot_data = category_data.pivot_table(
    values='Per_Capita_kg',
    index='Category',
    columns='Year',
    fill_value=0
)
pivot_data = pivot_data.sort_index()
fig = go.Figure(data=go.Heatmap(
    z=pivot_data.values,
    x=pivot_data.columns,
    y=pivot_data.index,
    colorscale='Plasma',
    colorbar_title='kg/capita/yr',
    name='Food Supply',
    hovertemplate=(
        '<b>Category: %{y}</b><br>' +
        'Year: %{x}<br>' +
        'Food Supply: %{z:.2f} kg/capita/yr<br>'
    ),
    hoverlabel=dict(
        font_color='black',
        bgcolor='white',
        bordercolor='black',
        font_size=12
    )
))
fig.update_layout(
    title='Per Capita Food Supply Across Categories and Years (2015–2022)',
    xaxis_title='Year',
    yaxis_title='Food Category',
    xaxis=dict(
        tickmode='linear',
        range=[2014.5, 2022.5]
    ),
    yaxis=dict(
        tickfont=dict(size=10),
        automargin=True
    ),
    plot_bgcolor='rgba(240, 240, 240, 0.9)',
    paper_bgcolor='rgba(255, 255, 255, 1)',
    font=dict(color='black', size=12),
    hovermode='closest',
    showlegend=False
)
fig.show()

#### **Proportional Per Capita Food Supply by Category**

In [24]:
yearly_data = df.groupby(['Year', 'Category'])['Per_Capita_kg'].sum().reset_index()
avg_data = df.groupby('Category')['Per_Capita_kg'].mean().reset_index()
avg_data['Year'] = 'Average'

fig = go.Figure()

years = sorted(yearly_data['Year'].unique())
categories = yearly_data['Category'].unique().tolist()
colors = px.colors.qualitative.Bold
for year in years:
    year_data = yearly_data[yearly_data['Year'] == year]
    labels = year_data['Category'].tolist()
    values = year_data['Per_Capita_kg'].tolist()

    fig.add_trace(
        go.Pie(
            labels=labels,
            values=values,
            textinfo='percent+label',
            textposition='auto',
            hovertemplate=(
                '<b>%{label}</b><br>' +
                'Food Supply: %{value:.2f} kg/capita/yr<br>' +
                'Percentage: %{percent:.2%}<br>'
            ),
            hoverlabel=dict(
                bgcolor='white',
                font_color='black',
                bordercolor='black',
                font_size=12
            ),
            marker=dict(
                colors=colors[:len(labels)],
                line=dict(color='white', width=1)
            ),
            pull=[0.02] * len(labels),
            rotation=45,
            sort=False,
            visible=(year == 2015)
        )
    )
avg_labels = avg_data['Category'].tolist()
avg_values = avg_data['Per_Capita_kg'].tolist()
fig.add_trace(
    go.Pie(
        labels=avg_labels,
        values=avg_values,
        textinfo='percent+label',
        textposition='auto',
        hovertemplate=(
            '<b>%{label}</b><br>' +
            'Food Supply: %{value:.2f} kg/capita/yr<br>' +
            'Percentage: %{percent:.2%}<br>'
        ),
        hoverlabel=dict(
            bgcolor='white',
            font_color='black',
            bordercolor='black',
            font_size=12
        ),
        marker=dict(
            colors=colors[:len(avg_labels)],
            line=dict(color='white', width=1)
        ),
        pull=[0.02] * len(avg_labels),
        rotation=45,
        sort=False,
        visible=False
    )
)
updatemenus = [
    dict(
        buttons=[
            dict(
                label=str(year),
                method='update',
                args=[{'visible': [i == j for i in range(len(years) + 1)]},
                      {'title': f'Proportional Per Capita Food Supply by Category in {year}'}]
            ) for j, year in enumerate(years)
        ] + [
            dict(
                label='Average (2015-2022)',
                method='update',
                args=[{'visible': [i == len(years) for i in range(len(years) + 1)]},
                      {'title': 'Average Food Supply Proportion by Category (2015-2022)'}]
            )
        ],
        direction='down',
        showactive=True,
        x=0.05,
        xanchor='left',
        y=1.1,
        yanchor='top'
    )
]
fig.update_layout(
    title='Proportional Per Capita Food Supply by Category in 2015',
    title_x=0.5,
    showlegend=True,
    legend=dict(
        orientation='v',
        x=1.1,
        xanchor='left',
        y=0.5,
        yanchor='middle',
        font=dict(size=10),
        bgcolor='rgba(255, 255, 255, 0.5)'
    ),
    plot_bgcolor='rgba(240, 240, 240, 0.9)',
    paper_bgcolor='white',
    font=dict(color='black', size=12),
    margin=dict(t=80, b=50, l=50, r=150),
    hovermode='closest',
    updatemenus=updatemenus
)
fig.show()

### **Merging Per Capita with Other Factors**

In [25]:
total_df = df.groupby('Year')['Per_Capita_kg'].sum().reset_index()
print(total_df)

   Year  Per_Capita_kg
0  2015         391.33
1  2016         402.58
2  2017         406.28
3  2018         407.26
4  2019         412.32
5  2020         420.92
6  2021         419.15
7  2022         415.83


In [26]:
merged2 = pd.read_csv('merged(2).csv')
# Merge on 'Year'
df = pd.merge(total_df, merged2, on='Year')
print(df)

   Year  Per_Capita_kg      Population  Total_Cropped_Area  Arable_Area  \
0  2015         391.33  210,969,298.00               23.55        30.99   
1  2016         402.58  213,524,840.00               23.13        31.21   
2  2017         406.28  216,379,655.00               23.53        31.16   
3  2018         407.26  219,731,479.00               22.65        30.53   
4  2019         412.32  223,293,280.00               24.16        30.93   
5  2020         420.92  227,196,741.00               24.00        30.28   
6  2021         419.15  231,402,117.00               23.85        30.23   

   Total_Irrigated_Area  Water_Availibility  Fertilizer_Consumption_Total  
0                 18.60              133.00                        3699.2  
1                 18.22              132.70                        5039.9  
2                 18.63              133.40                        4762.5  
3                 18.33              127.40                        4613.9  
4                 1

In [27]:
#Cleaning numeric columns that might contain commas and strings
for col in ['Population', 'Total_Cropped_Area', 'Arable_Area',
            'Total_Irrigated_Area', 'Water_Availibility',
            'Fertilizer_Consumption_Total']:
    df[col] = df[col].astype(str).str.replace(',', '').astype(float)

### **Data Visualizations**

#### **Total Per Capita Food Supply Over Time**

In [28]:
fig1 = px.line(df, x='Year', y='Per_Capita_kg', markers=True,
               title='Total Per Capita Food Supply Over Time (kg)',
               labels={'Per_Capita_kg': 'Per Capita (kg)'},template='plotly_white')
fig1.show()

#### **Trends in Agricultural and Environmental Factors Influencing Food Supply**

In [29]:
fig = make_subplots(
    rows=3, cols=2,
    shared_xaxes=True,
    subplot_titles=[
        'Population (count)',
        'Total Cropped Area (million ha)',
        'Arable Area (million ha)',
        'Irrigated Area (million ha)',
        'Water Availability (MAF)',
        'Fertilizer Consumption (\'000 tonnes)'
    ]
)
fig.add_trace(go.Scatter(x=df['Year'], y=df['Population'], name='Population',
                         mode='lines+markers'), row=1, col=1)

fig.add_trace(go.Scatter(x=df['Year'], y=df['Total_Cropped_Area'], name='Cropped Area',
                         mode='lines+markers'), row=1, col=2)

fig.add_trace(go.Scatter(x=df['Year'], y=df['Arable_Area'], name='Arable Area',
                         mode='lines+markers'), row=2, col=1)

fig.add_trace(go.Scatter(x=df['Year'], y=df['Total_Irrigated_Area'], name='Irrigated Area',
                         mode='lines+markers'), row=2, col=2)

fig.add_trace(go.Scatter(x=df['Year'], y=df['Water_Availibility'], name='Available Water',
                         mode='lines+markers'), row=3, col=1)

fig.add_trace(go.Scatter(x=df['Year'], y=df['Fertilizer_Consumption_Total'], name='Fertilizer Consumption',
                         mode='lines+markers'), row=3, col=2)
fig.update_layout(
    height=900,
    width=1200,
    title_text="Trends in Agricultural and Environmental Factors Influencing Food Supply (2015–2022)",
    showlegend=False,
    template='plotly_white'
)
fig.show()

#### **Correlation of Factors Influencing Per Capita Food Supply**

In [30]:
corr = df.drop(columns='Year').corr()
z = np.round(corr.values, 2)
heatmap = ff.create_annotated_heatmap(z=z, x=corr.columns.tolist(), y=corr.index.tolist(),
                                      colorscale='Viridis', showscale=True)
heatmap.update_layout(title='Correlation of Factors Influencing Per Capita Food Supply (2015–2022)')
heatmap.show()

### **Prediction and Modeling**

#### **Monte Carlo Simulation Using Linear Regression**

In [31]:
features = [
    'Year', 'Population', 'Total_Cropped_Area', 'Arable_Area',
    'Total_Irrigated_Area', 'Water_Availibility', 'Fertilizer_Consumption_Total'
]

X = df[features]
y = df['Per_Capita_kg']

model = LinearRegression()
model.fit(X, y)
feature_growth_stats = {}

for feature in features:
    growth = df[feature].pct_change()
    feature_growth_stats[feature] = {
        'mean': growth.mean(),
        'std': growth.std()
    }

In [41]:
n_years = 8
n_simulations = 1000
last_row = df[features].iloc[-1]
simulated_inputs = []

for sim in range(n_simulations):
    future_values = []
    current = last_row.copy()

    for year_offset in range(n_years):
        new_row = {}
        for feature in features:
            if feature == 'Year':
                new_row[feature] = 2023 + year_offset
            else:
                growth = np.random.normal(
                    loc=feature_growth_stats[feature]['mean'],
                    scale=feature_growth_stats[feature]['std']
                )
                current[feature] *= (1 + growth)
                new_row[feature] = current[feature]
        future_values.append(new_row)

    simulated_inputs.append(future_values)
simulated_results = []
for sim_set in simulated_inputs:
    df_sim = pd.DataFrame(sim_set)
    preds = model.predict(df_sim[features])
    simulated_results.append(preds)

future_years = np.arange(2023, 2023 + n_years)
simulated_results = np.array(simulated_results)

mean_forecast = simulated_results.mean(axis=0)
lower_bound = np.percentile(simulated_results, 5, axis=0)
upper_bound = np.percentile(simulated_results, 95, axis=0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Year'], y=df['Per_Capita_kg'],
                         mode='lines+markers', name='Historical'))

fig.add_trace(go.Scatter(x=future_years, y=mean_forecast, mode='lines+markers',
                         name='Monte Carlo Regression Forecast', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=future_years, y=upper_bound, mode='lines', line=dict(width=0), showlegend=False, name='90% Confidence Interval'))
fig.add_trace(go.Scatter(x=future_years, y=lower_bound, mode='lines',
                         fill='tonexty', fillcolor='rgba(255,165,0,0.2)',
                         line=dict(width=0), name='90% Confidence Interval'))

fig.update_layout(title='Monte Carlo + Regression Forecast of Per Capita Food Availability (2023–2030)',
                  xaxis_title='Year', yaxis_title='Per Capita Food (kg)', template='plotly_white', width=1300)

fig.show()

**📊 Interpretation of the Forecast Plot**


*   **Monte Carlo + Regression Forecast (Orange Line)**: The orange line represents the average predicted Per Capita Food Availability (kg) from 2023 to 2030 using Monte Carlo simulations based on a Linear Regression model trained on multiple input features (like population, water, land, fertilizer, etc.).
*   **Historical Data (Blue Line)**:
The blue line shows past recorded values from 2015 to 2022. The transition from blue to orange visually separates the observed from the forecasted period.
*   **90% Confidence Interval (Brown Shaded Area)**:
The shaded region represents the uncertainty band (5th to 95th percentile) around the forecasted values.
It widens after 2026, reflecting increasing prediction uncertainty—this is expected and realistic in Monte Carlo simulations.



> By 2030, the per capita availability is expected to lie between approximately **~425 kg and ~525 kg**, offering a practical planning window rather than a single deterministic estimate.



#####**Cross-validation for Monte Carlo + Linear Regression**

In [33]:
#3-Fold Cross Validation
kf = KFold(n_splits=3, shuffle=True, random_state=42)

r2_scores = []
rmse_scores = []

for train_idx, test_idx in kf.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2_scores.append(r2_score(y_test, y_pred))
    rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))

print(f"Average R² Score: {np.mean(r2_scores):.3f}")
print(f"Average RMSE: {np.mean(rmse_scores):.2f}")

Average R² Score: 0.223
Average RMSE: 5.52


### **XG Boost vs Naive Bayes**

####**XG Boost**

In [34]:
features = [
    'Year', 'Population', 'Total_Cropped_Area', 'Arable_Area',
    'Total_Irrigated_Area', 'Water_Availibility', 'Fertilizer_Consumption_Total'
]
X = df[features]
y = df['Per_Capita_kg']

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=features)

xgboost_model = XGBRegressor(
    max_depth=1, n_estimators=20, learning_rate=0.1, random_state=42
)
xgboost_model.fit(X_scaled, y)

y_pred_xgb = xgboost_model.predict(X_scaled)
xgb_r2 = r2_score(y, y_pred_xgb)
xgb_rmse = np.sqrt(mean_squared_error(y, y_pred_xgb))
xgb_mae = mean_absolute_error(y, y_pred_xgb)

print("XGBoost In-Sample Performance:")
print(f"  R²: {xgb_r2:.3f}, RMSE: {xgb_rmse:.2f}, MAE: {xgb_mae:.2f}")

# Monte Carlo simulation for forecasting
n_years = 8
n_simulations = 1000
last_row = X.iloc[-1].copy()
feature_growth_stats = {f: {'mean': df[f].pct_change().mean(), 'std': df[f].pct_change().std()} for f in features}

simulated_inputs_xgb = []
for sim in range(n_simulations):
    future_values = []
    current = last_row.copy()
    for year_offset in range(n_years):
        new_row = {}
        for feature in features:
            if feature == 'Year':
                new_row[feature] = 2023 + year_offset
            else:
                growth = np.random.normal(
                    loc=feature_growth_stats[feature]['mean'],
                    scale=feature_growth_stats[feature]['std']
                )
                current[feature] *= (1 + growth)
                new_row[feature] = current[feature]
        future_values.append(new_row)
    simulated_inputs_xgb.append(future_values)

simulated_results_xgb = []
for sim_set in simulated_inputs_xgb:
    df_sim = pd.DataFrame(sim_set)
    df_sim_scaled = pd.DataFrame(scaler.transform(df_sim[features]), columns=features)
    preds = xgboost_model.predict(df_sim_scaled)
    simulated_results_xgb.append(preds)

# Forecast results
future_years = np.arange(2023, 2023 + n_years)
simulated_results_xgb = np.array(simulated_results_xgb)
mean_forecast_xgb = simulated_results_xgb.mean(axis=0)
lower_bound_xgb = np.percentile(simulated_results_xgb, 5, axis=0)
upper_bound_xgb = np.percentile(simulated_results_xgb, 95, axis=0)

XGBoost In-Sample Performance:
  R²: 0.812, RMSE: 4.08, MAE: 2.79


**Interpretation:**
* R² = 0.871: The model explains 87.1% of the variance in Per_Capita_kg for the
full dataset (2015–2021, 7 points). This is a strong fit, indicating the model captures most of the trend (391.33 to 419.15 kg).
* RMSE = 3.76: The average prediction error is 3.76 kg/capita/year, reasonable given the target range (~391–420 kg).
* MAE = 2.91: The average absolute error is ~2.91 kg, suggesting predictions are typically off by ~3 kg, which is relatively small.

##### **Cross-Validation**

In [35]:
features = [
    'Year', 'Population', 'Total_Cropped_Area', 'Arable_Area',
    'Total_Irrigated_Area', 'Water_Availibility', 'Fertilizer_Consumption_Total'
]
X = df[features]
y = df['Per_Capita_kg']

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=features)

# 3-fold cross-validation
kf = KFold(n_splits=3, shuffle=True, random_state=42)
r2_scores = []
rmse_scores = []
mae_scores = []

for train_idx, test_idx in kf.split(X_scaled):
    X_train_cv, X_test_cv = X_scaled.iloc[train_idx], X_scaled.iloc[test_idx]
    y_train_cv, y_test_cv = y.iloc[train_idx], y.iloc[test_idx]

    xgboost_model = XGBRegressor(
        max_depth=1, n_estimators=20, learning_rate=0.1, random_state=42
    )
    xgboost_model.fit(X_train_cv, y_train_cv)

    y_pred_cv = xgboost_model.predict(X_test_cv)
    r2_scores.append(r2_score(y_test_cv, y_pred_cv))
    rmse_scores.append(np.sqrt(mean_squared_error(y_test_cv, y_pred_cv)))
    mae_scores.append(mean_absolute_error(y_test_cv, y_pred_cv))

print("XGBoost 3-Fold Cross-Validation Performance:")
print(f"  Mean R²: {np.mean(r2_scores):.3f} (±{np.std(r2_scores):.3f})")
print(f"  Mean RMSE: {np.mean(rmse_scores):.2f} (±{np.std(rmse_scores):.2f})")
print(f"  Mean MAE: {np.mean(mae_scores):.2f} (±{np.std(mae_scores):.2f})")

XGBoost 3-Fold Cross-Validation Performance:
  Mean R²: -0.044 (±0.611)
  Mean RMSE: 6.42 (±3.54)
  Mean MAE: 5.82 (±3.26)


#####**Interpretation:**
* Mean R² = -1.508: A negative R² means the model performs worse than predicting the mean Per_Capita_kg across the 3 folds. This suggests poor generalization, as the model fails to capture the trend in unseen data. The high standard deviation (±2.507) indicates unstable performance across folds, likely due to the small dataset (2–3 points per test fold).
* Mean RMSE = 8.30: The average CV prediction error is ~8.3 kg, significantly higher than in-sample (3.76 kg). The variability (±3.92) shows inconsistent errors across folds, reflecting sensitivity to small test sets.
* Mean MAE = 7.83: The average absolute error is ~7.83 kg, with high variability (±4.00), indicating predictions are often far off in CV.

####**Naive Bayes**

In [36]:
features = [
    'Year', 'Population', 'Total_Cropped_Area', 'Arable_Area',
    'Total_Irrigated_Area', 'Water_Availibility', 'Fertilizer_Consumption_Total'
]
X = df[features]
y = df['Per_Capita_kg']

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=features)

nb_model = GaussianNB()
y_discrete = np.round(y, 1).astype(int)
nb_model.fit(X_scaled, y_discrete)

y_pred_nb = nb_model.predict(X_scaled).astype(float)
nb_r2 = r2_score(y, y_pred_nb)
nb_rmse = np.sqrt(mean_squared_error(y, y_pred_nb))
nb_mae = mean_absolute_error(y, y_pred_nb)

print("Naive Bayes In-Sample Performance:")
print(f"  R²: {nb_r2:.3f}, RMSE: {nb_rmse:.2f}, MAE: {nb_mae:.2f}")

# Monte Carlo simulation
n_years = 8
n_simulations = 1000
last_row = X.iloc[-1].copy()
feature_growth_stats = {f: {'mean': df[f].pct_change().mean(), 'std': df[f].pct_change().std()} for f in features}

simulated_inputs_nb = []
for sim in range(n_simulations):
    future_values = []
    current = last_row.copy()
    for year_offset in range(n_years):
        new_row = {}
        for feature in features:
            if feature == 'Year':
                new_row[feature] = 2023 + year_offset
            else:
                growth = np.random.normal(
                    loc=feature_growth_stats[feature]['mean'],
                    scale=feature_growth_stats[feature]['std']
                )
                current[feature] *= (1 + growth)
                new_row[feature] = current[feature]
        future_values.append(new_row)
    simulated_inputs_nb.append(future_values)

simulated_results_nb = []
for sim_set in simulated_inputs_nb:
    df_sim = pd.DataFrame(sim_set)
    df_sim_scaled = pd.DataFrame(scaler.transform(df_sim[features]), columns=features)
    preds = nb_model.predict(df_sim_scaled).astype(float)
    simulated_results_nb.append(preds)

future_years = np.arange(2023, 2023 + n_years)
simulated_results_nb = np.array(simulated_results_nb)
mean_forecast_nb = simulated_results_nb.mean(axis=0)
lower_bound_nb = np.percentile(simulated_results_nb, 5, axis=0)
upper_bound_nb = np.percentile(simulated_results_nb, 95, axis=0)

Naive Bayes In-Sample Performance:
  R²: 0.997, RMSE: 0.47, MAE: 0.41


**Interpretation:**
* R² = 0.998: Near-perfect fit, explaining 99.8% of the variance. This is suspiciously high, suggesting severe overfitting, especially since Naive Bayes (GaussianNB) discretizes Per_Capita_kg (rounded to 1 decimal).
* RMSE = 0.47: Extremely low error (~0.47 kg), indicating predictions are almost identical to actual values in-sample.
* MAE = 0.40: Predictions are off by ~0.4 kg on average, reinforcing the near-perfect fit.

In [37]:
features = [
    'Year', 'Population', 'Total_Cropped_Area', 'Arable_Area',
    'Total_Irrigated_Area', 'Water_Availibility', 'Fertilizer_Consumption_Total'
]
X = df[features]
y = df['Per_Capita_kg']

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=features)

#3-fold cross-validation
kf = KFold(n_splits=3, shuffle=True, random_state=42)
r2_scores = []
rmse_scores = []
mae_scores = []

for train_idx, test_idx in kf.split(X_scaled):
    X_train_cv, X_test_cv = X_scaled.iloc[train_idx], X_scaled.iloc[test_idx]
    y_train_cv, y_test_cv = y.iloc[train_idx], y.iloc[test_idx]

    nb_model = GaussianNB()
    y_train_discrete = np.round(y_train_cv, 1).astype(int)
    nb_model.fit(X_train_cv, y_train_discrete)

    y_pred_cv = nb_model.predict(X_test_cv).astype(float)
    r2_scores.append(r2_score(y_test_cv, y_pred_cv))
    rmse_scores.append(np.sqrt(mean_squared_error(y_test_cv, y_pred_cv)))
    mae_scores.append(mean_absolute_error(y_test_cv, y_pred_cv))

print("Naive Bayes 3-Fold Cross-Validation Performance:")
print(f"  Mean R²: {np.mean(r2_scores):.3f} (±{np.std(r2_scores):.3f})")
print(f"  Mean RMSE: {np.mean(rmse_scores):.2f} (±{np.std(rmse_scores):.2f})")
print(f"  Mean MAE: {np.mean(mae_scores):.2f} (±{np.std(mae_scores):.2f})")

Naive Bayes 3-Fold Cross-Validation Performance:
  Mean R²: -0.719 (±1.782)
  Mean RMSE: 6.25 (±2.04)
  Mean MAE: 5.24 (±1.57)


**Interpretation**
* Mean R² = -3.887: Worse than XGBoost, indicating the model performs significantly worse than the mean predictor. The very high variability (±4.736) shows extreme instability across folds, likely due to discretization and violated assumptions (e.g., feature independence between Year, Population).
* Mean RMSE = 10.14: Higher error (~10.14 kg) than XGBoost (8.30 kg), with low variability (±0.57), suggesting consistently poor performance across folds.
* Mean MAE = 8.79: Higher than XGBoost (7.83 kg), with moderate variability (±1.25), indicating large and somewhat consistent errors.

#### **Conclusion: XGBoost vs. Naive Bayes**

* In-Sample:

 * XGBoost: R² = 0.871, RMSE = 3.76, MAE = 2.91.
 * Naive Bayes: R² = 0.998, RMSE = 0.47, MAE = 0.40.
 * Interpretation: Naive Bayes has a better in-sample fit, but its near-perfect metrics (R² = 0.998) indicate overfitting, as it essentially memorizes the training data. XGBoost’s more moderate fit (R² = 0.871) is still strong but suggests less overfitting, aligning better with a more realistic model.
* Cross-Validation:

 * XGBoost: Mean R² = -1.508 (±2.507), RMSE = 8.30 (±3.92), MAE = 7.83 (±4.00).
 * Naive Bayes: Mean R² = -3.887 (±4.736), RMSE = 10.14 (±0.57), MAE = 8.79 (±1.25).
* Interpretation:
 * R²: XGBoost’s less negative R² (-1.508 vs. -3.887) indicates it performs closer to the mean predictor than Naive Bayes, though both are poor. XGBoost’s lower variability (±2.507 vs. ±4.736) suggests more stable performance across folds.
 * RMSE: XGBoost’s lower RMSE (8.30 vs. 10.14) means smaller prediction errors, making it more accurate. However, XGBoost’s higher variability (±3.92 vs. ±0.57) shows less consistency across folds.
 * MAE: XGBoost’s lower MAE (7.83 vs. 8.79) reinforces its better accuracy, but high variability (±4.00 vs. ±1.25) indicates less consistent errors.

**Result:** XGBoost is more accurate in CV, with better R², RMSE, and MAE, though both models struggle with generalization due to the small dataset (7 points) and are not reliable for forecasting.